# Control Pannel

Please run the simulation by calling the launch file:  
`roslaunch final_assignment jupyter_simulation.launch`  
This will both start the simulation and run the essential nodes.

Let's start by importing the necessary modules:

In [ ]:
import jupyros as jr
import ipywidgets as widgets
from jupyros import ros3d
import os
from IPython.display import display
import rospy
import actionlib
from move_base_msgs.msg import *
from final_assignment.msg import CommandMessage
import numpy as np
import matplotlib.pyplot as plt
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from matplotlib.animation import FuncAnimation
import subprocess

Here we initialise the node, the command publisher and the action client:

In [ ]:
rospy.init_node('user_interface')
command = rospy.Publisher('/middleman/control', CommandMessage, queue_size=1)
client = actionlib.SimpleActionClient('move_base', MoveBaseAction)

The ToggleButtons widget allow to choose the different modalities:

In [ ]:
toggle = widgets.ToggleButtons(
    options=['Idle','Reach a point', 'Keyboard Driving', 'K. Assisted Driving'],
    description='Mode:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['The robot waits for a command',
              'Autonomously reach a specific point', 
              'Drive the robot with the keyboard', 
              'Drive the robot with the keyboard, assisted by the computer to avoid obstacles'],)

The two text boxes receive the coordinates for the robot to reach meanwhile, the go button is used to send the command:

In [ ]:
x = widgets.BoundedFloatText(
        value=0,
        min=-10,
        max=10,
        step=1,
        description='X:',
        disabled=True
    )

y = widgets.BoundedFloatText(
        value=0,
        min=-10,
        max=10,
        step=1,
        description='Y:',
        disabled=True
    )

go = widgets.Button(
        description='Go!',
        disabled=True,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Send the command!',
    )

Each time the ToggleButtons changes status, any active action is canceled and, if the current modality is anything but the 'Reach a point' function, the coordinates boxes are disabled. For every modality, the different parameters are set. The "Idle" modality re-set every counter and keeps the robot at rest waiting for a new working modality.
Every time the 'go' button is pushed, the counter of the requested goals is increased. Only when/if the point is reached, the reached goals counter is increased too. 
The 'go' button changes appearance when there's a target and gives the user the possibility of canceling the action.


In [ ]:
reached = 0 #Number of reached targets
total = 0   #Total number of requested targets
client.wait_for_server()
def on_click(change):
    global reached, total, x_data1, y_data1
    
    #Command for the middleman node
    control_command = CommandMessage()   
    control_command.enable_userCtrl = False
    control_command.enable_helper = False
    
    #Re-setting stuff
    client.cancel_all_goals()
    go.description='Go!'
    go.button_style='success'
    go.disabled=True
    x.disabled = True
    y.disabled = True    
    
    if change['new'] == 'Reach a point':
        x.disabled = False
        y.disabled = False
        go.disabled = False    
    elif change['new'] == 'Keyboard Driving':
        control_command.enable_userCtrl = True   
    elif change['new'] == 'K. Assisted Driving':
        control_command.enable_userCtrl = True     
        control_command.enable_helper = True
    else:
        x.value = 0 #Value of the coordinates box
        y.value = 0 #Value of the coordinates box
        reached = 0 
        total = 0
        x_data1, y_data1 = [], [] #Robot path
        
    command.publish(control_command) #Sending the command to the middleman node
        
toggle.observe(on_click, 'value')

def on_button_clicked(b):
    global total
    with output:
        if go.description=='Abort!': #If there's an active action and the 'go'/'abort' button is pushed
            client.cancel_all_goals()
            go.description='Go!'
            go.button_style='success'
        else:                         #If there's not an active action and the 'go'/'abort' button is pushed
            total = total + 1
            goal = MoveBaseGoal()
            goal.target_pose.header.frame_id = 'map'
            goal.target_pose.pose.orientation.w = 1.0
            goal.target_pose.pose.position.x = x.value
            goal.target_pose.pose.position.y = y.value
            client.send_goal(goal) 
            go.description='Abort!'
            go.button_style='danger'

go.on_click(on_button_clicked)

The node realizes the robot reached a target by checking its updated position in the `odom` topic. 
If it's within a range from the target (the odometry will almost never return the exact point), it will set the goal as reached and it will be ready for another goal. Actually, it won't stop right after it enters the virtual "circle" around the target: in order to give the robot the possibility of getting as much closer to the point as possible, it will need to satisfy the condition (of being inside the range) for a number of times before the script gives the target as reached.
Here the two vectors, where the path of the robot is saved, are also defined. This because it will be the odom_callback the function which appends the new position to these vectors.

In [ ]:
attempt=2000 #Number of times the robot has to be inside the range before the goal is considered reached

x_data1, y_data1 = [], []

def odom_callback(msg): #Each time the odometry is updated..
    global reached, attempt
    if go.description=='Abort!': #..check if the target is reached
        if msg.pose.pose.position.x<=x.value+1.2 and msg.pose.pose.position.x>=x.value-1.2 \
            and msg.pose.pose.position.y<=y.value+1.2 and msg.pose.pose.position.y>=y.value-1.2:
            #both x and y position must be within -1.2 and 1.2 units from the target
            if attempt<=0: #It is checked for a number of times so the robot can get as closer as possible
                reached = reached + 1
                client.cancel_all_goals()
                go.description='Go!'
                go.button_style='success'   
                attempt=2000
            else:
                attempt = attempt - 1
    #Update the robot path
    y_data1.append(msg.pose.pose.position.y)
    x_data1.append(msg.pose.pose.position.x) 

Once the callback is defined, it is finally possible to define also the subscriber to the `odom` topic:

In [ ]:
sub1 = rospy.Subscriber('/odom', Odometry, odom_callback)

The following widget is used together with the laser scanner plot. In the latter, the maximum radius is fixed to 10 units even though it is a higher distance than the average distance from an obstacle (in the studied environment). The slider allows to modify the maximum radius of the plot, allowing so to zoom in/out the showed graph.

In [ ]:
z = widgets.FloatSlider(
    value=5,
    min=1,
    max=10,
    step=0.5,
    description='Max radius:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',)

Here the main figure is defined. It will count 3 subplots:
- The real time path of the robot
- The laser scanner polar plot
- The reached targets/total targets bar chart  

The laser scanner is a polar plot because the distances returned by the topic are expressed in polar coordinates. Since the graph is uploaded with the most recent laser scans, the `scan` subscriber and its callback are coded. There is also the function "update_plot()" which is called for every new frame of the figure: it uploads each graph to make it show the updated data.
The "FuncAnimation" is called with the figure, the function to update the plots, the blit boolean and the calling interval as parameters.

In [ ]:
%matplotlib widget

fig = plt.figure()

fig.set_figheight(7)
fig.set_figwidth(9)

ax1 = plt.subplot(221)
ax4 = ax1 #this will plot an 'x' on the target
ax2 = plt.subplot(223, projection='polar')
ax3 = plt.subplot(222)
ln1, = ax1.plot([], [], 'b')
ln4, = ax1.plot([], [], 'xk')
ln2, = ax2.plot([], [], '.r')
ln3 = ax3.bar(['Reached','Total'],[reached, total], bottom=0)
ln = [ln1, ln2, ln3, ln4]    

ax1.set_title('Robot path')
ax1.set_xlim(-10, 10)
ax1.set_ylim(-10, 10)
ax1.set_xticks([-10, -5, 0, 5, 10])
ax1.set_yticks([-10, -5, 0, 5, 10])

ax2.set_title('Laser scans')
ax2.grid(True)
ax2.set_rmax(z.value)
ax2.set_thetamin(0)
ax2.set_thetamax(180)

ax2.set_xticks(np.pi/180. * np.linspace(0,  180, 5, endpoint=True))
ax2.set_xticklabels(['90°', '45°', '0°', '-45°', '-90°'])


x_data2 = np.linspace(0,np.pi,720) #720 angles between 0 and 180 degrees
y_data2 = np.linspace(10,10,720)

ax3.set_title('Goals outcomes')
ax3.set_ylim(-1, 11)
ax3.set_yticks([0,1,2,3,4,5,6,7,8,9,10])
ln3[0].set_color('g')
ln3[1].set_color('b')

def scan_callback(msg):
    global y_data2
    y_data2 = list(msg.ranges)
    
def update_plot(frame):
    plt.cla()
    if go.description=='Abort!': 
        ln[3].set_data(x.value, y.value) #Set an 'x' on the target
    else:                        
        ln[3].set_data([], [])           #Remove any old 'x' of the past targets
    #Update the plots
    ln[0].set_data(x_data1, y_data1)
    ln[1].set_data(x_data2, y_data2)
    ax2.set_rmax(z.value)             #Update the raiud according to the slider
    ln3 = ax3.bar(['Reached','Total'],[reached, total], bottom=0)
    ax3.set_title('Goals outcomes')
    ln3[0].set_color('g')
    ln3[1].set_color('b')
    ax3.set_ylim(-1, 11)
    ax3.set_yticks([0,1,2,3,4,5,6,7,8,9,10])
    ln[2] = ln3
    return ln

sub2 = rospy.Subscriber('/scan', LaserScan, scan_callback)

ani = FuncAnimation(fig, update_plot, blit=True, interval=1000)

#Plots positioning
plt.subplots_adjust(left=0.1,
                    bottom=0.01, 
                    right=0.95, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
#Button positioning
box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')
box = widgets.HBox(children=[go],layout=box_layout)

#Display
display(toggle)
display(x, y)
display(box)
output = widgets.Output()
display(output)
plt.show()
display(z)

Kill the simulation by running the following. 
NOTE: it will kill all the python code in execution on the machine

In [ ]:
#kill = subprocess.run(["killall", "-9", "python"])